[Reference](https://artificialcorner.com/insanely-fast-data-storytelling-with-chatgpt-and-python-1bddae3976f3)

In [1]:
import pandas as pd

# Load the CSV file
emissions_data = pd.read_csv("global_emissions.csv")

# Replace "USA" with "United States of America"
emissions_data["Country"] = emissions_data["Country"].replace("USA", "United States of America")

# Remove records with "Global" and "International Transport" from the Country field
emissions_data = emissions_data[~emissions_data["Country"].isin(["Global", "International Transport"])]

import plotly.graph_objects as go

filtered_data = emissions_data[(emissions_data["Year"] >= 1970) & (emissions_data["Year"] <= 2020)]

#get the top 10 countries by total emission
top_10_countries = filtered_data.groupby("Country")["Total"].mean().nlargest(10).index
plot_data = filtered_data[filtered_data["Country"].isin(top_10_countries)]
pivot_data = plot_data.pivot_table(values="Total", index="Year", columns="Country", aggfunc="sum", fill_value=0)

# create the graphic
fig_area = go.Figure()

# insert the data
for country in pivot_data.columns:
    fig_area.add_trace(go.Scatter(x=pivot_data.index, y=pivot_data[country], name=country, mode="none", stackgroup="one"))
fig_area.update_layout(title="Top 10 Countries by Total Emissions (1970-2020)", xaxis_title="Years", yaxis_title="Total Emissions")

#find average emissions by country
average_emissions = filtered_data.groupby("Country").agg({"Per Capita": "mean"})

#find the top 10 average emissions by country
top_10_per_capita = average_emissions.nlargest(10, "Per Capita").index

#display on stacked area and line chart
plot_data_per_capita = filtered_data[filtered_data["Country"].isin(top_10_per_capita)]
pivot_data_per_capita = plot_data_per_capita.pivot_table(values="Per Capita", index="Year", columns="Country", aggfunc="mean", fill_value=0)
fig_line = go.Figure()

for country in pivot_data_per_capita.columns:
    fig_line.add_trace(go.Scatter(x=pivot_data_per_capita.index, y=pivot_data_per_capita[country], name=country, mode="lines"))

fig_line.update_layout(title="Top 10 Countries by Per Capita Emissions (1970-2020)", xaxis_title="Years", yaxis_title="Per Capita Emissions")

import folium
import json

with open("world-countries.json", "r") as f:
    world_geo = json.load(f)

average_emissions_map_data = filtered_data.groupby("Country")["Per Capita"].mean().reset_index()
emissions_map = folium.Map(location=[0, 0], zoom_start=2)

folium.Choropleth(
    geo_data=world_geo,
    data=average_emissions_map_data,
    columns=["Country", "Per Capita"],
    key_on="feature.properties.name",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Average Per Capita Emissions"
).add_to(emissions_map)

emissions_map.save("map.html")

import dash
import dash_core_components as dcc
import dash_html_components as html

app = dash.Dash(__name__)

app.layout = html.Div([
    html.Iframe(srcDoc=open("map.html", "r").read(), width="100%", height="600"),
    html.Div([
        dcc.Graph(figure=fig_area, style={"width": "50%", "display": "inline-block"}),
        dcc.Graph(figure=fig_line, style={"width": "50%", "display": "inline-block"})
    ])
])

if __name__ == "__main__":
    app.run_server(debug=True)